In [1]:
# import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss


# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


In [2]:
# arguments
model_tokenizer = 'bert-base-uncased'
# do_lower_case=True


########################### Load dataset ###########################

In [3]:
# # import dataset 1 txt
# with open('../../DIIN/data/intrinsic_word_level_data/enwiki-20230320-pages-articles2.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df1 = pd.DataFrame(lines)[0]
# attribute_df1.reset_index(drop=True)
# attribute_df1

# # import dataset 2 txt
# with open('../../DIIN/data/intrinsic_word_level_data/enwiki-20230320-pages-articles3.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df2 = pd.DataFrame(lines)[0]
# attribute_df2.reset_index(drop=True)
# attribute_df2

# import dataset 3 txt
with open('../../DIIN/data/intrinsic_word_level_data/enwiki-20230320-pages-articles4.txt') as file:
    lines = [line.rstrip() for line in file]
attribute_df3 = pd.DataFrame(lines)[0]
attribute_df3.reset_index(drop=True)
attribute_df3

# # import dataset 4 txt
# with open('../../DIIN/data/enwiki-20230320-pages-articles5.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df4 = pd.DataFrame(lines)[0]
# attribute_df4.reset_index(drop=True)
# attribute_df4

# # import dataset 5 txt
# with open('../../DIIN/data/enwiki-20230320-pages-articles7.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df5 = pd.DataFrame(lines)[0]
# attribute_df5.reset_index(drop=True)
# attribute_df5

# # import dataset 6 txt
# with open('data/enwiki-20230320-pages-articles8.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df6 = pd.DataFrame(lines)[0]
# attribute_df6.reset_index(drop=True)
# attribute_df6

# # import dataset 7 txt
# with open('data/enwiki-20230320-pages-articles9.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df7 = pd.DataFrame(lines)[0]
# attribute_df7.reset_index(drop=True)
# attribute_df7

# # import dataset 8 csv
# attribute_df8 = pd.read_csv("data/jigsaw_train.csv")["comment_text"]

# merge all datasets and remove duplicates
# attribute_df = pd.concat([attribute_df1, attribute_df2, attribute_df3])
attribute_df = attribute_df3

# attribute_df0 = pd.read_csv("data/jigsaw_train.csv")["comment_text"]

In [4]:
attribute_df

0          Aripiprazole, sold under the brand names Abili...
1          In adults, side effects with greater than 10% ...
2          Common side effects include vomiting, constipa...
3          Aripiprazole was approved for medical use in t...
4          Aripiprazole is primarily used for the treatme...
                                 ...                        
1274140    In 1626 the 1st Viscount Grandison was also cr...
1274141    The second Viscount Grandison had had no sons ...
1274142    In 1746 Elizabeth Mason, daughter of John Vill...
1274143    Viscounts Grandison; First creation (1620; Rev...
1274144    "See Earl of Jersey for further Viscounts Gran...
Name: 0, Length: 1274145, dtype: object

########################### tokenize dataset ###########################

In [5]:
attribute_1 = "african"
female_attribute_list = [attribute_1]

attribute_2 = "european"
male_attribute_list = [attribute_2]

In [6]:

# test_input_ids, test_target_labels, test_attention_masks = tokenize(X_test)


# from transformers import BertTokenizer, AdamW, BertConfig, BertForPreTraining
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)
vocabs = tokenizer.get_vocab()
seq_len = 150


def tokenize_text(input_text):
    input_encoded_dict = tokenizer.batch_encode_plus(
                        list(input_text),                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = seq_len,           # Pad & truncate all sentences.
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    return input_encoded_dict.input_ids, input_encoded_dict.attention_mask

In [7]:
# from torch.utils.data import TensorDataset
# input_batch = list(attribute_df)[:int(0.9*len(attribute_df))]
# val_input_batch = list(attribute_df)[int(0.9*len(attribute_df)):]

# input_ids, input_attention_mask = tokenize_text(input_batch)
# val_input_ids, val_input_attention_mask = tokenize_text(val_input_batch)

# train_dataset_1 = TensorDataset(input_ids, input_attention_mask)
# val_dataset_1 = TensorDataset(val_input_ids, val_input_attention_mask)

# torch.save(train_dataset_1, "1_train_dataset3.pt")
# torch.save(val_dataset_1, "1_val_dataset3.pt")

In [8]:
# train_dataset_1 = torch.load("1_train_dataset.pt")
# val_dataset_1 = torch.load("1_val_dataset.pt")

In [9]:
# train_dataset_1[0][0]
# # jbfjbfjb j

In [10]:

def get_attributes_ids(gender_attributes):
    gender_attributes_ids = []
    for j in gender_attributes:
#         toks = tokenizer1(sentence).input_ids
        wordpieces = tokenizer(j).input_ids
#         print(j, len(wordpieces))
        if len(wordpieces)==3:
            gender_attributes_ids.append(wordpieces[1])
            # print(j)
    return gender_attributes_ids
        
        
        
def tokenize(tokenized_data, max_token_count):
    # Load the BERT tokenizer.
    print('Loading BERT tokenizer...')

    # Training Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    stereotype_token_id = []
    stereotype_token_index = []
    gender_label = []
    attention_masks = []
    count = {}
    
    # get male and female stereopyes
    # train_female_attributes_ids = get_attributes_ids(female_attribute_list[:int(0.8*len(female_attribute_list))])
    # train_male_attributes_ids = get_attributes_ids(male_attribute_list[:int(0.8*len(male_attribute_list))])
    train_female_attributes_ids = get_attributes_ids(female_attribute_list)
    train_male_attributes_ids = get_attributes_ids(male_attribute_list)
    print(train_female_attributes_ids, "\n")
    print(train_male_attributes_ids)
    
    for i in train_female_attributes_ids+train_male_attributes_ids:
        count[i] = 0
    
    # For every sentence...
    for k, sent in enumerate(tokenized_data):
        tokens_ids = sent[0].cpu().numpy().copy()
        att_masks = sent[1].cpu().numpy().copy()
        # print(tokens_ids)
        

        #         target_output = tokens.copy()
        if (count[train_female_attributes_ids[0]]>= max_token_count) and (count[train_male_attributes_ids[0]]>= max_token_count):
            print("iii")
            break
        else:
            for i, token in enumerate(tokens_ids):
                if token in train_female_attributes_ids:
                    if count[token]<max_token_count: #prevent over imbalance of tokens
                        tokens_tensor = torch.tensor([tokens_ids])
                        input_ids.append(tokens_tensor)
                        attention_masks.append(torch.tensor([att_masks]))
                        stereotype_token_id.append(token)
                        stereotype_token_index.append(i)
                        gender_label.append(0)
                        count[token] += 1
                elif token in train_male_attributes_ids:
                    if count[token]<max_token_count:
                        tokens_tensor = torch.tensor([tokens_ids])
                        input_ids.append(tokens_tensor)
                        attention_masks.append(torch.tensor([att_masks]))
                        stereotype_token_id.append(token)
                        stereotype_token_index.append(i)
                        gender_label.append(1)
                        count[token] += 1        
        
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
#     print(input_ids)
    attention_masks = torch.cat(attention_masks, dim=0)
#     print(attention_masks)
    stereotype_token_id = torch.tensor(stereotype_token_id)
    
    stereotype_token_index = np.array(stereotype_token_index)
    b = np.zeros((stereotype_token_index.size, seq_len))
    b[np.arange(stereotype_token_index.size),stereotype_token_index] = 1
    stereotype_token_index = torch.tensor(b.astype(int))
#     print(stereotype_token_index)
    
    gender_label = torch.tensor(gender_label)

    return input_ids, attention_masks,  stereotype_token_id, stereotype_token_index, gender_label, count

In [17]:
# max_token_count = 100000
input_ids = torch.tensor(np.array([]).astype(int))
attention_masks = torch.tensor(np.array([]).astype(int))
stereotype_token_id = torch.tensor(np.array([]).astype(int))
stereotype_token_index = torch.tensor(np.array([]).astype(int))
gender_label = torch.tensor(np.array([]).astype(int))

val_input_ids = torch.tensor(np.array([]).astype(int))
val_attention_masks = torch.tensor(np.array([]).astype(int))
val_stereotype_token_id = torch.tensor(np.array([]).astype(int))
val_stereotype_token_index = torch.tensor(np.array([]).astype(int))
val_gender_label = torch.tensor(np.array([]).astype(int))


for ii in range(1,4):
    train_dataset_1 = torch.load("1_train_dataset"+str(ii)+".pt")
    val_dataset_1 = torch.load("1_val_dataset"+str(ii)+".pt")
    max_token_count =20000
    input_idsii, attention_masksii,  stereotype_token_idii, stereotype_token_indexii, gender_labelii, count = tokenize(train_dataset_1, max_token_count = max_token_count)
    val_input_idsii, val_attention_masksii,  val_stereotype_token_idii, val_stereotype_token_indexii, val_gender_labelii, val_count = tokenize(val_dataset_1, max_token_count = int(max_token_count/10))

    input_ids = torch.cat((input_ids, input_idsii), 0)
    attention_masks = torch.cat((attention_masks, attention_masksii), 0)
    stereotype_token_id = torch.cat((stereotype_token_id, stereotype_token_idii), 0)
    stereotype_token_index = torch.cat((stereotype_token_index, stereotype_token_indexii), 0)
    gender_label = torch.cat((gender_label, gender_labelii), 0)


    val_input_ids = torch.cat((val_input_ids, val_input_idsii), 0)
    val_attention_masks = torch.cat((val_attention_masks, val_attention_masksii), 0)
    val_stereotype_token_id = torch.cat((val_stereotype_token_id, val_stereotype_token_idii), 0)
    val_stereotype_token_index = torch.cat((val_stereotype_token_index, val_stereotype_token_indexii), 0)
    val_gender_label = torch.cat((val_gender_label, val_gender_labelii), 0)
    
    # print('Original: ', X_train[0])
    print('input_ids: ', input_ids[0], "\n")
    print('gender_label:', gender_label[0], "\n")
    print('stereotype_token_id: ', stereotype_token_id[0], "\n")
    print('stereotype_token_index:', stereotype_token_index[0], "\n")
    print('gender_label:', gender_label[0], "\n")
    
    
    from torch.utils.data import TensorDataset
    
    train_dataset_2 = TensorDataset(input_ids, attention_masks,  stereotype_token_id, stereotype_token_index, gender_label)
    val_dataset_2 = TensorDataset(val_input_ids, val_attention_masks,  val_stereotype_token_id, val_stereotype_token_index, val_gender_label)
    # test_dataset = TensorDataset(test_input_ids, test_target_labels, test_attention_masks)
    
    print('{:>5,} training samples'.format(len(train_dataset_2)))
    print('{:>5,} validation samples'.format(len(val_dataset_2)))
    print('count', count)
    print('validation count', val_count)
    # print('{:>5,} test samples'.format(len(test_dataset)))


Loading BERT tokenizer...
[3060] 

[2647]
Loading BERT tokenizer...
[3060] 

[2647]
input_ids:  tensor([  101,  1037,  2261,  4111,  2427,  2024,  1996,  2765,  1997,  8893,
         3989,  1012,  1996,  8840,  8713,  6906,  4875,  2003,  1037,  3019,
         8893,  1012,  1996,  2137,  2417,  4702,  3544,  2000,  2022,  1037,
         8893,  1997,  1996,  3897,  4702,  1998,  1996, 20457,  1010,  2348,
         2049, 27691,  3570,  2038,  2042,  1037,  3395,  1997,  6704,  1012,
         1996,  2647, 21006, 10729,  2003,  1037,  4100,  1011,  4568,  8893,
         2090,  4770, 17582,  1998,  9409, 17582,  1025,  2049,  2313,  5942,
         1996,  2506,  3739,  1997,  2012,  2560,  2028,  1997,  1996,  6687,
         2427,  1012,  5430,  5265,  5769,  2008,  1996,  2647, 22285,  2003,
         1037,  3019,  8893,  1997,  1996,  8740,  3217, 18069,  1998,  1996,
        29096, 22285,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    

########################### save dataset ###########################

In [18]:
# save tokenized dataset
torch.save(train_dataset_2, '2_train_'+attribute_1+'_'+attribute_2+'_tokens.pt')
torch.save(val_dataset_2, '2_val_'+attribute_1+'_'+attribute_2+'_tokens.pt')

In [13]:
# len(input_ids)

In [14]:
# max_token_count = 100000
# max_token_count = 15000
# input_ids, attention_masks,  stereotype_token_id, stereotype_token_index, gender_label, count = tokenize(train_dataset_1, max_token_count = max_token_count)
# val_input_ids, val_attention_masks,  val_stereotype_token_id, val_stereotype_token_index, val_gender_label, val_count = tokenize(val_dataset_1, max_token_count = int(max_token_count/10))
# # test_input_ids, test_target_labels, test_attention_masks = tokenize(X_test)

# # print('Original: ', X_train[0])
# print('input_ids: ', input_ids[0], "\n")
# print('gender_label:', gender_label[0], "\n")
# print('stereotype_token_id: ', stereotype_token_id[0], "\n")
# print('stereotype_token_index:', stereotype_token_index[0], "\n")
# print('gender_label:', gender_label[0], "\n")


# from torch.utils.data import TensorDataset

# train_dataset_2 = TensorDataset(input_ids, attention_masks,  stereotype_token_id, stereotype_token_index, gender_label)
# val_dataset_2 = TensorDataset(val_input_ids, val_attention_masks,  val_stereotype_token_id, val_stereotype_token_index, val_gender_label)
# # test_dataset = TensorDataset(test_input_ids, test_target_labels, test_attention_masks)

# print('{:>5,} training samples'.format(len(train_dataset_2)))
# print('{:>5,} validation samples'.format(len(val_dataset_2)))
# print('count', count)
# print('validation count', val_count)
# # print('{:>5,} test samples'.format(len(test_dataset)))
